In [ ]:
#!/usr/bin/env python
"""
Drip Campaign Automation Script

This script:
- Reads an Excel file (assumed synced locally from OneDrive) containing email details.
- Extracts first names from email addresses and constructs personalized emails.
- Connects to Outlook to send the emails.
- Updates the Excel file with the email sent status.
- Checks for email responses.
- Drives 5 rounds of drip email follow-ups on scheduled days.
- Applies conditional formatting to flag responses (this sample shows how to mark responses; 
  you might later want to enhance this for visual formatting).

Important: Adjust file paths, email body templates, and scheduling parameters as needed.
"""

import os
import re
import time
import random
import datetime
import schedule  # For scheduling tasks
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from win32com.client import Dispatch

# ---------------------
# Global Configurations
# ---------------------

# Path to the synced OneDrive Excel File
EXCEL_FILE_PATH = r"C:\Users\YourUsername\OneDrive\CampaignData.xlsx"

# Outlook email account settings
OUTLOOK_ACCOUNT = "your_outlook_email@domain.com"

# Email subject and body template; use {first_name} placeholder for personalization.
EMAIL_SUBJECT = "Your Campaign Subject Here"
EMAIL_BODY_TEMPLATE = """
Hello {salutation},

This is your drip campaign email body.
Please let us know if you have any questions.

Best regards,
Your Sales Team
"""

# Status columns in the Excel file (assumed fixed positions/headers)
# Modify the header names/index as per your actual Excel layout
EMAIL_COLUMN_HEADER = "Email"
FIRST_SENT_HEADER = "First Email Sent"  # Mark when round 1 is sent
SECOND_SENT_HEADER = "Second Email Sent"  # Round 2 follow-up
THIRD_SENT_HEADER = "Third Email Sent"    # Round 3 follow-up
FOURTH_SENT_HEADER = "Fourth Email Sent"  # Round 4 follow-up
FIFTH_SENT_HEADER = "Fifth Email Sent"    # Round 5 follow-up
RESPONSE_HEADER = "Response"  # To hold response text

# Conditional formatting settings (for demonstration purposes)
RESPONSE_FILL = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")

# Rounds scheduling (in days from initial send)
ROUND_DAYS = [0, 2, 5, 9, 13]  # D0: first send, then round 2 on Day 3, day 6 etc.
# Note: The schedule always uses Day 0 as the starting point.

# -------------
# UTIL FUNCTIONS
# -------------

def extract_first_name(email_address):
    """
    Extracts the first name from an email address.

    If the extracted name is less than 2 characters or non-alphabetic,
    returns 'dear team'
    """
    if not email_address:
        return "dear team"
    # Assume email is like "firstname.lastname@domain.com"
    local_part = email_address.split('@')[0]
    # Split by non-alphabetic characters (like dot, underscore, etc.)
    parts = re.split(r"[^a-zA-Z]+", local_part)
    first_name = parts[0].strip() if parts and parts[0] else ""
    if len(first_name) < 2 or not first_name.isalpha():
        return "dear team"
    return first_name.capitalize()

def get_workbook_sheet(path):
    """
    Loads the Excel workbook and returns the active sheet.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Excel file not found at: {path}")
    workbook = load_workbook(path)
    sheet = workbook.active  # assuming the active sheet contains the data
    return workbook, sheet

def find_header_indices(sheet):
    """
    Reads the first row (header) of the Excel sheet and maps header names to column indices.

    Returns a dictionary with header names (keys) and corresponding column letters/index.
    """
    header_map = {}
    for col in range(1, sheet.max_column + 1):
        header_value = sheet.cell(row=1, column=col).value
        if header_value:
            header_map[header_value.strip()] = col
    return header_map

def update_cell(sheet, row, col, value):
    """
    Updates the given cell in the sheet.
    """
    sheet.cell(row=row, column=col).value = value

def save_workbook(workbook, path):
    """
    Saves the workbook back to the specified path.
    """
    workbook.save(path)

# --------------------
# OUTLOOK EMAIL SENDER
# --------------------

def send_outlook_email(recipient, subject, body):
    """
    Sends an email via Outlook COM interface.
    """
    try:
        outlook = Dispatch("Outlook.Application")
        mail = outlook.CreateItem(0)
        mail.To = recipient
        mail.Subject = subject
        mail.Body = body
        mail.Send()
        print(f"[{datetime.datetime.now()}] Email sent to: {recipient}")
        return True
    except Exception as e:
        print(f"Error while sending email to {recipient}: {e}")
        return False

# ----------------------------
# EMAIL CAMPAIGN CORE LOGIC
# ----------------------------

def process_campaign_row(sheet, row, header_map, round_number):
    """
    Process a single row in the Excel sheet for a given round of the campaign.
    - Extract email address.
    - Check if an email has already been sent for the current round.
    - Extract first name for personalization.
    - Format email and send.
    - Update sheet status if email was sent.
    """
    email = sheet.cell(row=row, column=header_map[EMAIL_COLUMN_HEADER]).value
    if not email:
        return

    # Determine the appropriate sent column header for this round.
    round_header = {
        1: FIRST_SENT_HEADER,
        2: SECOND_SENT_HEADER,
        3: THIRD_SENT_HEADER,
        4: FOURTH_SENT_HEADER,
        5: FIFTH_SENT_HEADER,
    }.get(round_number)

    if round_header not in header_map:
        print(f"Warning: Sent column '{round_header}' not found. Skipping row {row}.")
        return

    # Check if email already sent for this round (if cell has a value)
    sent_flag = sheet.cell(row=row, column=header_map[round_header]).value
    if sent_flag is not None:
        # Email already sent in this round
        return

    # Get first name and set salutation
    first_name = extract_first_name(email)
    salutation = first_name if first_name.lower() != "dear team" else "dear team"

    # Format email content
    email_body = EMAIL_BODY_TEMPLATE.format(first_name=first_name,
                                              salutation=salutation)

    # Send the email
    sent_success = send_outlook_email(email, EMAIL_SUBJECT, email_body)
    if sent_success:
        # Update the sent column with current timestamp
        update_cell(sheet, row, header_map[round_header], 
                    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    else:
        print(f"Failed to send email for row {row}")

def process_campaign_round(round_number):
    """
    Processes one round of the drip campaign.
    Loads the workbook, iterates over each row that needs to be processed for the given round,
    sends emails, then saves the workbook.
    """
    print(f"Starting campaign round {round_number}...")
    workbook, sheet = get_workbook_sheet(EXCEL_FILE_PATH)
    header_map = find_header_indices(sheet)

    # Iterate over rows, skipping header row (start at row = 2)
    for row in range(2, sheet.max_row + 1):
        # If a response has already been received, skip sending further follow-ups.
        response = sheet.cell(row=row, column=header_map.get(RESPONSE_HEADER, -1)).value
        if response:
            continue

        process_campaign_row(sheet, row, header_map, round_number)

    # Save workbook post processing
    save_workbook(workbook, EXCEL_FILE_PATH)
    print(f"Round {round_number} processed and workbook updated.")

# ----------------------------
# RESPONSE PROCESSING FUNCTION
# ----------------------------

def check_and_update_responses():
    """
    Connects to Outlook and checks for replies to your campaign emails.
    It uses a simple filter on unread messages.
    For every email, it identifies the sender, extracts the text body,
    and writes it to the 'Response' column in the workbook at the matching row.
    
    NOTE: In a production system, you may need a more robust matching mechanism.
    """
    print("Checking for email responses...")
    try:
        outlook = Dispatch("Outlook.Application")
        ns = outlook.GetNamespace("MAPI")
        inbox = ns.GetDefaultFolder(6)  # 6 refers to the inbox folder

        messages = inbox.Items
        messages = messages.Restrict("[UnRead] = True")  # Only process unread emails

        workbook, sheet = get_workbook_sheet(EXCEL_FILE_PATH)
        header_map = find_header_indices(sheet)

        for message in messages:
            try:
                # Get the sender's email address.
                sender = message.SenderEmailAddress
                response_body = message.Body
                # Loop through workbook rows to find matching email (case-insensitive)
                for row in range(2, sheet.max_row + 1):
                    row_email = sheet.cell(row=row, column=header_map[EMAIL_COLUMN_HEADER]).value
                    if row_email and row_email.lower() == sender.lower():
                        # Update the response cell with the email response
                        update_cell(sheet, row, header_map[RESPONSE_HEADER], response_body)
                        # Apply conditional formatting (visual marker) by setting a background fill.
                        sheet.cell(row=row, column=header_map[RESPONSE_HEADER]).fill = RESPONSE_FILL
                        print(f"Updated response for {sender}")
                        break
                # Mark the email as read in Outlook
                message.UnRead = False
            except Exception as inner_e:
                print(f"Error processing message: {inner_e}")

        save_workbook(workbook, EXCEL_FILE_PATH)
        print("Response processing completed and workbook updated.")
    except Exception as e:
        print(f"Error connecting to Outlook for response checking: {e}")

# ------------------------
# SCHEDULING AND MAIN LOOP
# ------------------------

def schedule_campaign():
    """
    Schedules the drip campaign rounds and response checking over time.
    Rounds:
      - Round 1: Immediately (Day 1)
      - Round 2: After 2 days from round 1 (i.e., Day 3)
      - Round 3: After 5 days from round 1 (i.e., Day 6)
      - Round 4: After 9 days from round 1 (i.e., Day 10)
      - Round 5: After 13 days from round 1 (i.e., Day 14)

    This function demonstrates scheduling using the schedule module.
    For random time scheduling within a window, the function schedules a job at a random
    minute within a specified hour. Modify as needed.
    """
    # Schedule each round:
    start_time = datetime.datetime.now()
    round_delays = ROUND_DAYS  # delays in days relative to the first send

    for idx, delay in enumerate(round_delays, start=1):
        scheduled_time = start_time + datetime.timedelta(days=delay)
        # For randomization: pick a random second within the day window.
        random_seconds = random.randint(0, 60 * 60 * 24 - 1)
        scheduled_datetime = scheduled_time + datetime.timedelta(seconds=random_seconds)
        delay_seconds = (scheduled_datetime - datetime.datetime.now()).total_seconds()
        if delay_seconds < 0:
            delay_seconds = 0  # In case of scheduling in the past

        # Use threading.Timer to schedule function execution
        from threading import Timer
        Timer(delay_seconds, process_campaign_round, args=(idx,)).start()
        print(f"Scheduled Round {idx} for {scheduled_datetime}")

    # Schedule regular response checks every hour
    schedule.every(60).minutes.do(check_and_update_responses)

def run_schedule_loop():
    """
    Runs the scheduling loop to process scheduled tasks.
    """
    while True:
        schedule.run_pending()
        time.sleep(10)

# ------------------
# MAIN ENTRY POINT
# ------------------

def main():
    """
    Main function to kickoff the drip campaign:
    - Schedule campaign rounds.
    - Start periodic response checking.
    - Start scheduling loop.
    """
    print("Starting Drip Campaign...")
    schedule_campaign()
    run_schedule_loop()

if __name__ == "__main__":
    main()
